In [1]:
import numpy as np
import math
from bounding_box_utils.bounding_box_utils import iou, convert_coordinates
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy, match_multi
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
from bs4 import BeautifulSoup

In [5]:
fh = open('/home/nassar/gits/datasets/VOC/Pasadena/ImageSets/Main/norm/trainval_standard.txt','r')
counter = 0
for x in fh:
    with open('/home/nassar/gits/datasets/VOC/Pasadena/Annotations/'+x[:-1]+'.xml') as f:
        soup = BeautifulSoup(f, 'xml')
        if soup.find('yaw'):
            print("yaw: ",soup.yaw.text)
        else:
            print("EEEEEEEEEEEEEEEEEEEEE")
        

yaw:  0.9086184085882479
yaw:  6.252642045269685
yaw:  3.1278045524990383
yaw:  0.0143116998663535
yaw:  3.134785869507015
yaw:  0.017976891295541593
yaw:  1.5505505074717623
yaw:  2.529505684915382
yaw:  4.646241001734104
yaw:  1.558928087881335
yaw:  0.021642082724729686
yaw:  3.475474139496308
yaw:  6.181432611788317
yaw:  4.684638245277981
yaw:  0.3803072440095644
yaw:  6.272364265817221
yaw:  3.114365517258682
yaw:  3.1257101573966444
yaw:  1.51738925168387
yaw:  1.5273376284202378
yaw:  6.256132703773673
yaw:  2.260724980108255
yaw:  1.2716468930030684
yaw:  0.39426987802551905
yaw:  1.4196508135721877
yaw:  2.3513075682867606
yaw:  6.057514234896719
yaw:  1.5330972149518192
yaw:  2.8213247358488336
yaw:  5.823640115129479
yaw:  3.2121039553703636
yaw:  1.5437437233889846
yaw:  3.141243587739394
yaw:  3.1361821329086106
yaw:  4.667184952758037
yaw:  2.8982937558617836
yaw:  1.570447260944498
yaw:  1.5514231720977594
yaw:  1.5552628964521469
yaw:  3.519107370796166
yaw:  1.5320500

yaw:  3.1539844912789525
yaw:  6.255609104998077
yaw:  2.5609216114512794
yaw:  0.01832595714594046
yaw:  0.013264502315156903
yaw:  4.125783818789396
yaw:  6.23902847710413
yaw:  1.5599752854325317
yaw:  1.8530160668423798
yaw:  4.102570939737872
yaw:  6.246707925812904
yaw:  6.2625904220060535
yaw:  1.496445300659938
yaw:  5.8622118915985535
yaw:  6.268000942687236
yaw:  4.700869807321526
yaw:  3.1098276612034965
yaw:  6.270968002415627
yaw:  1.4795156069155933
yaw:  1.5407766636605942
yaw:  4.714483375487083
yaw:  6.270968002415627
yaw:  3.1560788863813465
yaw:  1.5498523757709646
yaw:  4.715006974262681
yaw:  2.7665213973362115
yaw:  1.5646876744129163
yaw:  0.6012659273120465
yaw:  3.110351259979095
yaw:  4.717624968140673
yaw:  1.5107570005262918
yaw:  2.4172810140121466
yaw:  1.5973253314252103
yaw:  1.5624187463853236
yaw:  1.6662658368789862
yaw:  2.559874413900083
yaw:  3.1452578450189814
yaw:  5.701641600415075
yaw:  5.899736470516431
yaw:  5.622578185299732
yaw:  2.94995550

yaw:  4.815887005027954
yaw:  4.323878688890752
yaw:  1.553692100125352
yaw:  2.8213247358488336
yaw:  0.33807027611130164
yaw:  3.0707322859588233
yaw:  3.115238181884679
yaw:  4.703662334124719
yaw:  6.278298385274003
yaw:  1.0471975511965976
yaw:  3.152588227877357
yaw:  3.143163449916588
yaw:  6.266953745136038
yaw:  0.826413400819315
yaw:  0.4134684997974567
yaw:  6.278647451124401
yaw:  5.94668582739508
yaw:  3.0756192078644076
yaw:  1.5507250403969617
yaw:  0.182212373908208
yaw:  1.0705849631733217
yaw:  2.444857216193657
yaw:  0.014835298641951801
yaw:  4.711516315758693
yaw:  3.166550861893312
yaw:  3.1342622707314174
yaw:  2.2001620550640517
yaw:  1.5580554232553376
yaw:  3.045599544730105
yaw:  1.7034413499464653
yaw:  3.12413936106985
yaw:  4.714657908412282
yaw:  6.259972428128061
yaw:  3.141592653589793
yaw:  0.3844960342143508
yaw:  6.216862795603802
yaw:  4.701567939022325
yaw:  1.5678292670665062
yaw:  4.695982885415943
yaw:  3.1248374927706473
yaw:  3.826983450847966

yaw:  6.2650338829588454
yaw:  3.050486466635689
yaw:  5.159716867670836
yaw:  1.5631168780861215
yaw:  2.835810968640386
yaw:  2.5223498349822053
yaw:  4.702091537797924
yaw:  3.1255356244714454
yaw:  0.9976302004399586
yaw:  4.705756729227111
yaw:  4.692666759837153
yaw:  3.12099776841626
yaw:  1.5098843359002947
yaw:  5.732708461100574
yaw:  4.20432363512914
yaw:  6.134134189059271
yaw:  2.879095134089846
yaw:  4.704185932900316
yaw:  4.95848040491589
yaw:  2.4619614428632013
yaw:  1.566083937814512
yaw:  6.264859350033646
yaw:  3.1017991466443227
yaw:  5.7079247857222555
yaw:  6.272713331667619
yaw:  1.5629423451609221
yaw:  1.5575318244797396
yaw:  1.1854276279545486
yaw:  0.15690509975429023
yaw:  3.3545228223331014
yaw:  4.684289179427581
yaw:  4.062428366942001
yaw:  4.057541445036417
yaw:  0.4628613176288962
yaw:  3.1140164514082826
yaw:  4.693713957388351
yaw:  1.5533430342749535
yaw:  0.04258603374866164
yaw:  1.4830062654195817
yaw:  3.045599544730105
yaw:  3.16602726311771

yaw:  2.568077461384456
yaw:  1.5517722379481584
yaw:  1.570621793869697
yaw:  3.106337002699507
yaw:  1.5203563114122605
yaw:  1.5632914110113207
yaw:  1.666614902729385
yaw:  3.1202996367154623
yaw:  3.1365311987590094
yaw:  4.718148566916271
yaw:  1.5625932793105233
yaw:  3.1332150731802204
yaw:  4.85358611687103
yaw:  3.134960402432215
yaw:  1.4337879805133418
yaw:  0.003316125578789226
yaw:  3.1185543074634676
yaw:  4.7006952743963275
yaw:  3.161663939987728
yaw:  0.00890117918517108
yaw:  1.5924384095196262
yaw:  4.69703008296714
yaw:  3.078935333443196
yaw:  6.181432611788317
yaw:  1.5631168780861215
yaw:  3.167772592369708
yaw:  0.10053096491487339
yaw:  1.5468853160425742
yaw:  1.714611457159229
yaw:  1.5425219929125884
yaw:  6.281090912077193
yaw:  3.873932807726614
yaw:  6.271666134116423
yaw:  3.160267676586132
yaw:  6.262241356155654
yaw:  1.5486306452945686
yaw:  3.1017991466443227
yaw:  0.6382669074543263
yaw:  0.4003785304074992
yaw:  1.203753585100489
yaw:  1.521927107

...

yaw:  0.826413400819315
yaw:  2.5071654704898547
yaw:  5.9093357814024
yaw:  3.1079077990263024
yaw:  6.264161218332848
yaw:  2.5085617338914497
yaw:  3.105987936849109
yaw:  3.1011010149435245
yaw:  4.673119072214817
yaw:  6.261019625679258
yaw:  0.054105206811824215
yaw:  2.6506315350037886
yaw:  4.713436177935886
yaw:  4.754800481208152
yaw:  1.5522958367237565
yaw:  5.120272426575765
yaw:  0.0005235987755982988
yaw:  4.689699700108763
yaw:  2.2205824073123854
yaw:  3.068463357931231
yaw:  3.129026282975434
yaw:  4.125783818789396
yaw:  6.263288553706851
yaw:  3.1309461451526275
yaw:  1.5596262195821329
yaw:  6.271142535340825
yaw:  3.1176816428374705
yaw:  3.7383207248466546
yaw:  6.278123852348803
yaw:  1.538333202707802
yaw:  1.5524703696489561
yaw:  6.274109595069215
yaw:  0.2638937829015426
yaw:  1.5142476590302802
yaw:  1.1995647948957027
yaw:  6.273062397518019
yaw:  4.694935687864747
yaw:  6.261194158604458
yaw:  1.9997982569351027
yaw:  4.6952847537151445
yaw:  3.0707322859

yaw:  3.1314697439282257
yaw:  0.17889624832941875
yaw:  3.114714583109081
yaw:  6.048962121561948
yaw:  6.271317068266025
yaw:  0.5677556056737554
yaw:  4.181983420703613
yaw:  1.5667820695153096
yaw:  6.273585996293617
yaw:  4.726526147325844
yaw:  3.1079077990263024
yaw:  3.13024801345183
yaw:  3.943396911955988
yaw:  4.689001568407966
yaw:  2.6789058688860967
yaw:  6.264510284183246
yaw:  3.143861581617386
yaw:  3.1344368036566164
yaw:  4.032757769658097
yaw:  1.5323990832510213
yaw:  2.2999948882781274
yaw:  6.273585996293617
yaw:  6.2460097941121075
yaw:  1.578650308428871
yaw:  1.728225025324785
yaw:  3.1264082890974425
yaw:  4.843986805985063
yaw:  1.7311920850531755
yaw:  3.132691474404622
yaw:  4.854284248571829
yaw:  1.7188002473640156
yaw:  6.2788219840496
yaw:  0.011693705988362009
yaw:  4.667708551533635
yaw:  0.9559168313172943
yaw:  1.5456635855661782
yaw:  4.702440603648323
yaw:  4.6799258562975945
yaw:  1.5498523757709646
yaw:  6.164677450969171
yaw:  5.94843115664707

yaw:  3.135134935357414
yaw:  3.1710887179484972
yaw:  0.5115560037595379
yaw:  3.1087804636522995
yaw:  3.123266696443852
yaw:  5.870065873232528
yaw:  4.041658948843269
yaw:  4.016351674689351
yaw:  5.8622118915985535
yaw:  2.1907372771032825
yaw:  1.5180873833846678
yaw:  4.731238536306228
yaw:  0.08674286382411817
yaw:  3.131818809778625
yaw:  3.6964328227987906
yaw:  2.527585822738188
yaw:  1.5578808903301387
yaw:  3.1138419184830832
yaw:  1.5552628964521469
yaw:  1.618443815374342
yaw:  1.5666075365901102
yaw:  6.268873607313233
yaw:  1.5392058673337994
yaw:  6.002187297608499
yaw:  4.722511890046256
yaw:  1.576206847476079
yaw:  4.743804906920588
yaw:  1.6509069394614364
yaw:  6.251420314793289
yaw:  2.719921106307963
yaw:  4.685161844053578
yaw:  4.687081706230772
yaw:  3.373023312404241
yaw:  6.26957173901403
yaw:  3.0998792844671286
yaw:  1.5442673221645826
yaw:  1.5198327126366622
yaw:  5.568298445562709
yaw:  1.5550883635269477
yaw:  0.3609340893124273
yaw:  2.7527332962454

yaw:  3.1258846903218442
yaw:  1.5648622073381158
yaw:  2.9234264970905017
yaw:  6.281439977927592
yaw:  3.132342408554223
yaw:  4.679402257521997
yaw:  0.42027528388023455
yaw:  4.695633819565544
yaw:  2.526364092261792
yaw:  6.272015199966823
yaw:  6.237108614926935
yaw:  2.296504229774139
yaw:  3.134960402432215
yaw:  6.2749822596952125
yaw:  4.687954370856769
yaw:  3.1199505708650634
yaw:  4.85358611687103
yaw:  3.1000538173923284
yaw:  2.8391270942191755
yaw:  6.272538798742421
yaw:  3.132342408554223
yaw:  3.0928979674591517
yaw:  2.5054201412378605
yaw:  6.273236930443219
yaw:  3.137927462160605
yaw:  1.5803956376808654
yaw:  4.671897341738421
yaw:  0.06335545184739416
yaw:  0.4626867847036968
yaw:  3.306526267903257
yaw:  1.6105898337403675
yaw:  6.055419839794326
yaw:  1.5655603390389135
yaw:  1.5531685013497536
yaw:  4.703662334124719
yaw:  6.037792014349183
yaw:  6.254736440372079
yaw:  2.9675833271659586
yaw:  6.271840667041623
yaw:  1.5676547341413067
yaw:  5.5272832081408

yaw:  1.5505505074717623
yaw:  1.3177235852557188
yaw:  5.221152457341036
yaw:  5.324301416133902
yaw:  1.5594516866569332
yaw:  3.1394982584873996
yaw:  3.143687048692186
yaw:  2.7665213973362115
yaw:  1.5535175672001527
yaw:  5.867273346429338
yaw:  0.026878070480712675
yaw:  2.261074045958654
yaw:  1.5503759745465628
yaw:  4.713785243786285
yaw:  3.1267573549478413
yaw:  3.1394982584873996
yaw:  5.573534433318692
yaw:  6.264684817108446
yaw:  4.692492226911955
yaw:  6.270269870714828
yaw:  1.5776031108776747
yaw:  0.15638150097869194
yaw:  3.135134935357414
yaw:  5.1200978936505654
yaw:  0.915076126820627
yaw:  0.13840460968315033
yaw:  3.134960402432215
yaw:  2.7403414585562964
yaw:  0.3356268151585096
yaw:  3.1225685647430548
yaw:  5.7112409113010445
yaw:  3.9306560084164297
yaw:  4.848350129115048
yaw:  4.0050070345513875
yaw:  4.694586622014348
yaw:  4.738917985015004
yaw:  5.709844647899448
yaw:  3.144036114542585
yaw:  4.684289179427581
yaw:  3.036872898470133
yaw:  4.11688263